## Initialization

In [68]:
import torch, argparse
import numpy as np

import os, sys

# sys.path.append(os.path.abspath(os.path.join('..')))

THIS_DIR = os.path.abspath(os.path.join('.'))
PARENT_DIR = os.path.dirname(THIS_DIR)
sys.path.append(PARENT_DIR)

from nn_models import MLP
from hnn import HNN
from utils import L2_loss, rk4
from data_position_only import get_dataset

In [69]:
print(THIS_DIR)
print(PARENT_DIR)

/Users/ZongyuWu/hamiltonian-nn/experiment-spring
/Users/ZongyuWu/hamiltonian-nn


## Help Functions

In [70]:
# print squared loss at specific steps for comparison with HNN
print_every = 200
def print_results(results, print_every=200):
    for step in range(0, len(results["train_loss"]), print_every):
        print(
            "step {}, train_loss {:.4e}, test_loss {:.4e}".format(
                step,
                results["train_loss"][step],
                results["test_loss"][step],
            )
        )
    # print('Final train loss {:.4e} +/- {:.4e}\nFinal test loss {:.4e} +/- {:.4e}'
    #     .format(results["train_loss"][-1], results["train_std"][-1],
    #             results["test_loss"][-1], results["test_std"][-1]))

def print_best(results):
    curr_min = 0

    for step in range(0, len(results["train_loss"])):
        if results["test_loss"][step] < results["test_loss"][curr_min]:
            curr_min = step
    print(
        "best test loss at step {}, train_loss {:.4e}, test_loss {:.4e}".format(
            curr_min,
            results["train_loss"][curr_min],
            results["test_loss"][curr_min],
        )
    )

In [71]:

def get_args():
    parser = argparse.ArgumentParser(description=None)
    parser.add_argument(
        "--input_dim", default=2, type=int, help="dimensionality of input tensor"
    )
    parser.add_argument(
        "--hidden_dim", default=200, type=int, help="hidden dimension of mlp"
    )
    parser.add_argument("--learn_rate", default=1e-3, type=float, help="learning rate")
    parser.add_argument(
        "--nonlinearity", default="tanh", type=str, help="neural net nonlinearity"
    )
    parser.add_argument(
        "--total_steps", default=2000, type=int, help="number of gradient steps"
    )
    parser.add_argument(
        "--print_every",
        default=200,
        type=int,
        help="number of gradient steps between prints",
    )
    parser.add_argument(
        "--name", default="spring", type=str, help="only one option right now"
    )
    parser.add_argument(
        "--baseline",
        dest="baseline",
        action="store_true",
        help="run baseline or experiment?",
    )
    parser.add_argument(
        "--use_rk4",
        dest="use_rk4",
        action="store_true",
        help="integrate derivative with RK4",
    )
    parser.add_argument(
        "--verbose", dest="verbose", action="store_true", help="verbose?"
    )
    parser.add_argument(
        "--kan", dest="kan", action="store_true", help="use kan instead of mlp?"
    )
    parser.add_argument(
        "--field_type",
        default="solenoidal",
        type=str,
        help="type of vector field to learn",
    )
    parser.add_argument("--seed", default=0, type=int, help="random seed")
    parser.add_argument(
        "--save_dir", default=THIS_DIR, type=str, help="where to save the trained model"
    )
    parser.set_defaults(feature=True)
    return parser.parse_args()

In [72]:
def train(args):
    # set random seed
    torch.manual_seed(args.seed)
    np.random.seed(args.seed)

    # init model and optimizer
    if args.verbose:
        print("Training baseline model:" if args.baseline else "Training HNN model:")

    output_dim = args.input_dim if args.baseline else 2
    nn_model = MLP(args.input_dim, args.hidden_dim, output_dim, args.nonlinearity)
    model = HNN(
        args.input_dim,
        differentiable_model=nn_model,
        field_type=args.field_type,
        baseline=args.baseline,
    )
    optim = torch.optim.Adam(model.parameters(), args.learn_rate, weight_decay=1e-4)

    # arrange data
    data = get_dataset(seed=args.seed)
    x = torch.tensor(data["x"], requires_grad=True, dtype=torch.float32)
    test_x = torch.tensor(data["test_x"], requires_grad=True, dtype=torch.float32)
    dxdt = torch.Tensor(data["dx"])
    test_dxdt = torch.Tensor(data["test_dx"])

    # vanilla train loop
    stats = {"train_loss": [], "test_loss": []}
    for step in range(args.total_steps + 1):

        # train step
        dxdt_hat = (
            model.rk4_time_derivative(x) if args.use_rk4 else model.time_derivative(x)
        )
        loss = L2_loss(dxdt, dxdt_hat)
        loss.backward()
        optim.step()
        optim.zero_grad()

        # run test data
        test_dxdt_hat = (
            model.rk4_time_derivative(test_x)
            if args.use_rk4
            else model.time_derivative(test_x)
        )
        test_loss = L2_loss(test_dxdt, test_dxdt_hat)

        # logging
        stats["train_loss"].append(loss.item())
        stats["test_loss"].append(test_loss.item())
        if args.verbose and step % args.print_every == 0:
            print(
                "step {}, train_loss {:.4e}, test_loss {:.4e}".format(
                    step, loss.item(), test_loss.item()
                )
            )

    train_dxdt_hat = model.time_derivative(x)
    train_dist = (dxdt - train_dxdt_hat) ** 2
    test_dxdt_hat = model.time_derivative(test_x)
    test_dist = (test_dxdt - test_dxdt_hat) ** 2
    print(
        "Final train loss {:.4e} +/- {:.4e}\nFinal test loss {:.4e} +/- {:.4e}".format(
            train_dist.mean().item(),
            train_dist.std().item() / np.sqrt(train_dist.shape[0]),
            test_dist.mean().item(),
            test_dist.std().item() / np.sqrt(test_dist.shape[0]),
        )
    )

    return model, stats

## Create Dataset

In [73]:
# create dataset
data = get_dataset(seed=0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.tensor(data["x"], requires_grad=True, dtype=torch.float32).to(device)
test_x = torch.tensor(data["test_x"], requires_grad=True, dtype=torch.float32).to(device)
dxdt = torch.Tensor(data["dx"]).to(device)
test_dxdt = torch.Tensor(data["test_dx"]).to(device)

# dataset['train_input'], dataset['train_label'],dataset['test_input'], dataset['test_label']
dataset = {
    "train_input": x,
    "train_label": dxdt,
    "test_input": test_x,
    "test_label": test_dxdt,
}

In [74]:
dataset["train_input"].shape, dataset["train_label"].shape

(torch.Size([7500, 2]), torch.Size([7500, 2]))

## Model Training

In [75]:
args = get_args()
args.verbose = True
model, stats = train(args)

# save
os.makedirs(args.save_dir) if not os.path.exists(args.save_dir) else None
label = "-position"
path = "{}/{}{}.tar".format(args.save_dir, args.name, label)
torch.save(model.state_dict(), path)


Training HNN model:
step 0, train_loss 2.0823e+00, test_loss 2.0830e+00
step 200, train_loss 3.1658e-03, test_loss 4.6407e-03
step 400, train_loss 2.9592e-03, test_loss 4.4936e-03
step 600, train_loss 2.9248e-03, test_loss 4.4616e-03
step 800, train_loss 2.9073e-03, test_loss 4.4440e-03
step 1000, train_loss 2.8952e-03, test_loss 4.4319e-03
step 1200, train_loss 2.8859e-03, test_loss 4.4228e-03
step 1400, train_loss 2.8780e-03, test_loss 4.4147e-03
step 1600, train_loss 2.9854e-03, test_loss 4.4085e-03
step 1800, train_loss 2.8700e-03, test_loss 4.4014e-03
step 2000, train_loss 2.8858e-03, test_loss 4.4076e-03
Final train loss 2.8716e-03 +/- 6.2804e-04
Final test loss 4.4076e-03 +/- 1.0590e-03


In [76]:
print_results(stats, 1)
print_best(stats)

step 0, train_loss 2.0823e+00, test_loss 2.0830e+00
step 1, train_loss 2.0625e+00, test_loss 2.0699e+00
step 2, train_loss 2.0498e+00, test_loss 2.0538e+00
step 3, train_loss 2.0341e+00, test_loss 2.0366e+00
step 4, train_loss 2.0170e+00, test_loss 2.0204e+00
step 5, train_loss 2.0009e+00, test_loss 2.0053e+00
step 6, train_loss 1.9860e+00, test_loss 1.9900e+00
step 7, train_loss 1.9710e+00, test_loss 1.9736e+00
step 8, train_loss 1.9549e+00, test_loss 1.9561e+00
step 9, train_loss 1.9377e+00, test_loss 1.9379e+00
step 10, train_loss 1.9199e+00, test_loss 1.9192e+00
step 11, train_loss 1.9016e+00, test_loss 1.8997e+00
step 12, train_loss 1.8824e+00, test_loss 1.8786e+00
step 13, train_loss 1.8618e+00, test_loss 1.8557e+00
step 14, train_loss 1.8392e+00, test_loss 1.8306e+00
step 15, train_loss 1.8145e+00, test_loss 1.8035e+00
step 16, train_loss 1.7878e+00, test_loss 1.7742e+00
step 17, train_loss 1.7588e+00, test_loss 1.7424e+00
step 18, train_loss 1.7273e+00, test_loss 1.7076e+00
ste

![hnn](./hnn.png)
![baseline](./baseline.png)
